# Classify Clothes into 24 classes Using CNN
> author: LHK    
> date: 29/07/2021

## Import Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense,AveragePooling2D,Lambda
from tensorflow.keras.models import  Model
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Nadam, Adam
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
from PIL import Image, ImageOps
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tensorflow as tf
import torch
from keras.models import Sequential
torch.cuda.empty_cache()
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from os.path import dirname, basename
import glob
import random
from keras.applications.resnet50 import ResNet50
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam,SGD,RMSprop
from sklearn.preprocessing import LabelBinarizer
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

## Load given csv file && dip into it

In [ ]:
# df = pd.read_csv('')

## Load dataset 
* Load images (resize to [90,90] ) and labels from dir
* Divive 8/1/1 || Train-Valid-Test 
* Generate images in Train Set by ImageDataGenerator

In [ ]:
X_train = []
y_train = []

X_valid = []
y_valid = []

X_test = []
y_test = []

count_train = []
count_valid = []
count_test = []

SIZE_IMAGE = [90,90]

gen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.01, zoom_range=0.1, 
            channel_shift_range=10., horizontal_flip=True)

labels = glob.glob( '../input/triplesmarket/dataset5/dataset/*/')
list_labels = [basename(dirname(lbl)) for lbl in labels]

for idx in range(len(list_labels)):
    image_paths = glob.glob('../input/triplesmarket/dataset5/dataset/'+str(list_labels[idx])+'/*.jpg')
    random.shuffle(image_paths)
    c_train=0
    c_valid=0
    c_test=0
    gate = 0
    for path in image_paths[:int(len(image_paths)*.2)]:
        if gate%2 == 0:
            c_valid+=1
            imagee = np.array(Image.open(path).resize(SIZE_IMAGE))
            X_valid.append(imagee)
            y_valid.append(str(list_labels[idx]))
        if gate%2 != 0:
            c_test+=1
            imagee = np.array(Image.open(path).resize(SIZE_IMAGE))
            X_test.append(imagee)
            y_test.append(str(list_labels[idx]))
        gate +=1

    for path in image_paths[int(len(image_paths)*.2):]:
        
        imagee = Image.open(path).resize(SIZE_IMAGE)
        
        image = np.expand_dims(imagee,0)
        aug_iter = gen.flow(image)
        
        aug_images = [next(aug_iter)[0].astype(np.uint8) for i in range(10)]
        for img in aug_images:
            c_train+=1
            X_train.append(img)
            y_train.append(str(list_labels[idx]))
    count_train.append(c_train)
    count_valid.append(c_valid)
    count_test.append(c_test)

### Visualize data set

In [ ]:
fig = plt.figure(figsize=(10, 10))
index = 1
x = random.randint(0,len(X_train)-13)
for i in range(x, x+12):
    fig.add_subplot(3, 4, index)
    fig.tight_layout()
    plt.imshow(X_train[i])
    index+=1
plt.show()

In [ ]:
%matplotlib inline

x = np.arange(len(list_labels))  
width = 0.3 

fig, ax = plt.subplots(figsize=(20,7))
fig. tight_layout(pad=3.0)

rects1 = ax.bar(x  , [x//10 for x in count_train], width, label='Train')
rects2 = ax.bar(x + width , count_valid, width, label='Valid')
rects3 = ax.bar(x + (width*2), count_test, width, label='Test')

ax.set_title('Train-Valid-Test before Generate')
ax.set_xticks(x)
ax.set_xticklabels(list_labels,rotation = 45, ha="right")
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)
ax.bar_label(rects3, padding=3)

plt.show()

### Random show sample image in train set

In [ ]:
idx = random.randint(0,len(X_train))
plt.imshow(X_train[idx])
plt.title(y_train[idx])

### Random shuffle dataset

In [ ]:
data_train = list(zip(X_train,y_train))

random.shuffle(data_train)
X_train_ran = []
y_train_ran = []
for val in data_train :
    X_train_ran.append(val[0])
    y_train_ran.append(val[1])

    
data_valid = list(zip(X_valid,y_valid))

random.shuffle(data_valid)
X_valid_ran = []
y_valid_ran = []
for val in data_valid :
    X_valid_ran.append(val[0])
    y_valid_ran.append(val[1])
    
data_test = list(zip(X_test,y_test))

random.shuffle(data_test)
X_test_ran = []
y_test_ran = []
for val in data_test :
    X_test_ran.append(val[0])
    y_test_ran.append(val[1])

In [ ]:
for i in range(len(X_train_ran)):
    image_without_alpha_t = X_train_ran[i][:,:,:3]
    X_train_ran[i] = image_without_alpha_t
    
for i in range(len(X_valid_ran)):
    image_without_alpha_v = X_valid_ran[i][:,:,:3]
    X_valid_ran[i] = image_without_alpha_v
    
for i in range(len(X_test_ran)):
    image_without_alpha_v = X_test_ran[i][:,:,:3]
    X_test_ran[i] = image_without_alpha_v

### Change label in Train&Valid set to int

In [ ]:
label = {
    "ao-len-cardigan": 0,
    "quan-nu": 1,
    "quan-nam": 2,
    "dam-nu": 3,
    "ao-nguc": 4,
    "bo-vest-nu": 5,
    "ao-nu": 6,
    "trang-phuc-cosplay": 7,
    "ao-dai": 8,
    "do-boi-nam": 9,
    "vay-chong-nang-khau-trang": 10,
    "do-hoa-trang-trang-phuc-cosplay-nam": 11,
    "ao-polo-nam": 12,
    "thoi-trang-nu-quan-dai": 13,
    "quan-jogger-nam": 14,
    "ao-len-nam": 15,
    "dam-dao-pho": 16,
    "quan-vo-nu": 17,
    "do-lot-do-ngu-nam": 18,
    "ao-hoodie-nu": 19,
    "ao-vest-blazer-nu": 20,
    "ao-khoac-kieu-nam": 21,
    "do-mac-nha": 22,
    "ao-thun-ba-lo": 23
}

In [ ]:
for i in range(len(y_train_ran)):
    y_train_ran[i] = label[y_train_ran[i]]
    
for i in range(len(y_valid_ran)):
    y_valid_ran[i] = label[y_valid_ran[i]]
    
for i in range(len(y_test_ran)):
    y_test_ran[i] = label[y_test_ran[i]]

In [ ]:
catLB = LabelBinarizer()
catLabels_train = catLB.fit_transform(y_train_ran)
catLabels_valid = catLB.fit_transform(y_valid_ran)

In [ ]:
catLabels_train

In [ ]:
x_train_stack = np.stack(X_train_ran)
y_train_stack = np.stack(catLabels_train)

x_valid_stack = np.stack(X_valid_ran)
y_valid_stack = np.stack(catLabels_valid)

In [ ]:
class FashionNet:
    @staticmethod
    def build_category_model(inputs,numCat):
        
        x = Conv2D(32, (3, 3),strides=2, padding="same",activation='relu')(inputs)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        
        x = Conv2D(64, (3, 3),strides=1, padding="same",activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dropout(0.6)(x)
        x = Conv2D(64, (3, 3), padding="same",activation='relu')(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        x = Dropout(0.5)(x)
        
        x = Conv2D(128, (3, 3), padding="same",activation='relu',kernel_regularizer =tf.keras.regularizers.l2( l=0.01))(x)
        x = Dropout(0.4)(x)
        x = BatchNormalization()(x)
        
        x = Conv2D(256, (3, 3), padding="same",activation='relu',kernel_regularizer =tf.keras.regularizers.l2( l=0.01))(x)
        x = Dropout(0.1)(x)
        x = BatchNormalization()(x)
        x = MaxPooling2D(pool_size=(3, 3))(x)
        
        x = Flatten()(x)
        x = Dense(256, activation='relu')(x)
        x = BatchNormalization()(x)
        x = Dense(numCat, activation='softmax',name='category_output')(x)
        
        return x
    @staticmethod
    def build(height,width,channels,numCat):
        inputShape = (height,width,channels)
        
        inputs = Input(shape=inputShape)
        
        categoryBranch = FashionNet.build_category_model(inputs,numCat)
        
        model = Model(inputs=inputs , outputs=[categoryBranch],name='fashionnet')
        
        return model

In [ ]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 128

In [ ]:
model = FashionNet.build(90,90,3,numCat=len(catLB.classes_))

losses = {
 "category_output": "categorical_crossentropy",
}
lossWeights = {"category_output": 1.0}

opt = Adam(lr=INIT_LR, decay=INIT_LR/EPOCHS)

model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights,metrics=["accuracy"])

print(model.summary())

keras_callbacks   = [
#           EarlyStopping( patience=20, verbose=1, restore_best_weights=False),
    
#           ReduceLROnPlateau( factor=0.1, patience=10, min_lr=1e-6, verbose=1),
    
          ModelCheckpoint('model/sss.hdf5',monitor = 'val_loss',save_best_only = True, verbose = 1 )
        ]

In [ ]:
from collections import Counter
itemCt = Counter(y_train_ran)
maxCt = float(max(itemCt.values()))
cw = {clsID : maxCt/numImg for clsID, numImg in itemCt.items()}

In [ ]:
cw

In [ ]:
from sklearn.utils import class_weight


history = model.fit(x_train_stack,y_train_stack,
              validation_data=(x_valid_stack,y_valid_stack),
              steps_per_epoch=len(x_train_stack)//BS,
              shuffle = True,
              epochs=EPOCHS,
              validation_steps=len(x_valid_stack)//BS,
              batch_size=BS,
              callbacks=[keras_callbacks],
              class_weight=cw)

In [ ]:
model = load_model('model/sss.hdf5')

In [ ]:
y_hat = [np.argmax(model.predict(np.array([img]))) for img in X_test_ran]

In [ ]:
from sklearn.metrics import f1_score
f1_score(y_test_ran, y_hat, average='micro')

In [ ]:
fig = plt.figure(figsize=(14, 14))
columns = 5
rows = 5
for i in range(1, columns*rows +1):
    idx = random.randint(0,len(X_test)-1)
    img = X_test[idx]
    y_hat = np.argmax(model.predict(np.array([img])))
    lbl = ''
    truth = ''
    for key, value in label.items():
        if str(y_hat) == str(value):
            lbl = key
        if str(y_test_ran[idx]) == str(value):
            truth = key
    fig.add_subplot(rows, columns, i)
    fig.tight_layout()
    plt.imshow(img)
    plt.title(lbl+'/'+str(y_test[idx]))
plt.show()

In [ ]:
from matplotlib.pyplot import figure

figure(figsize=(10, 8), dpi=80)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 5])
plt.legend(loc='lower right')

figure(figsize=(10, 8), dpi=80)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy') 
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0 ,1])
plt.legend(loc='lower right')

## Calculate Top-N Accuracy (1;3;5)

In [ ]:
y_hat = []
for img in X_test_ran:
    tmp = model.predict(np.array([img]))
    y_hat.append(tmp)
y_hat = np.array(y_hat)

In [ ]:
def arg_sort_desc(ar):
    P = np.argsort(ar)
    Q=[]
    for i in range(P.shape[0]):
        p=list(P[i,:][::-1])
        Q.append(p)
    R=np.array(Q)
    return(P)

def top_n_accuracy(A,C,n):
    B = arg_sort_desc(C)
    topn = B[:,:,:n]
    A = np.array(A)
    return np.mean(np.array([1 if A[k] in topn[k] else 0 for k in range(len(topn))]))

In [ ]:
top_n_accuracy(y_test_ran,y_hat,5)